# Cost Functions

Now we'll discuss cost functions.  Recall that we need them to train our networks.  We will have essentially two, depending on whether we're doing a regression or classification problem.

## Mean Squared Error

A classical cost function is the *mean square error*.  This is essentially what is sounds like.  If we predict $(\hat y_1, \ldots, \hat y_m)$ but the right answer was $(y_1,\ldots,y_m)$, then our error is a *vector*, which is $(y_1-\hat y_1, \ldots, y_m - \hat y_m)$.

To get the mean squared error, we square all of these, then take the mean.  That is: $$\textrm{MSE}=\frac{1}{m}\sum_{i=1}^m (y_m-\hat y_m)^2$$

It's convenient to compute the derivative of this, as well.  Recall that what we actually want is the derivative of the cost with respect to $\hat y$.  We can consider each row as a separate approximation of the cost, so we'll compute the derivative with respect to $\hat y$ for each summand (row), then take the mean.

The derivative of $(y_i-\hat y_i)^2$ with regard to $\hat y_m$ is exactly $2\cdot (y_i-\hat y_i)$.  It's easy to code:

In [1]:
def cost_MSE(y_hat, y, diff=False, aggregate=False):
    if diff:
        return 2 * (y_hat-y)
    elif aggregate:
        # sum-square each row, then take the mean across all rows
        return np.mean(np.sum((y-y_hat)**2, axis=1), axis=0)
    else:
        # sum-square each row
        return np.sum((y-y_hat)**2, axis=1)

It's important to see a big difference between what happens when the flags are triggered.  If `diff` is on, we want the derivative.  What this means is that we want the derivative, with respect to every neuron (hence all the columns), with respect to every row.  This is a big grid of errors, but contains all the information we'll need.

If `diff` and `aggregate` are both off, it just gives the error for each row, which is the formula you'd expect above.

If `aggregate` is on (and `diff` is off), then it takes the average.  This is not important for the training process, but it gives a single number indicating our progress, so could be useful for human interpretation of how the training is going, as well as (potentially) meta-training algorithms, where we could alter the learning rate depending on the cost, for example.

In any case, this is a great cost function for regression, which admittedly is not a commonly studied use for neural networks.  However, it's less well-used for classification problems, for reasons we're about to describe.

## The Cross Entropy Cost Function

Suppose we're doing a classification problem, so the prediction vector will be (strictly) between 0 and 1, and the things you're trying to predict are vectors of zeros and ones.  This is important to making this well-defined.  But first, let's motivate it.

Recall that the motion of the weights is proportional to both the derivative of the cost function, and the derivative of the activation function.  The derivative of the cost function (with MSE) is exactly proportional to the amount of error, which sounds good.  However, if the correct answer was zero and our prediction was near one (or vice-versa), then because our prediction was near an edge, the derivative of the activation function is near zero (assuming we're using a sigmoid or hyperbolic tangent function).  The product will then be approximately zero, and the network will learn slowly, despite the large error.

The fix is to make it so that if the error is near one (that is, very confident and wrong), then the derivative of the cost function should be near infinite, so that the product of the error functions will be sizeable.  Since we're doing a classification problem, the following is well-defined:

$$C(\hat y, y)=-y\ln(\hat y)-(1-y)\ln(1-\hat y)$$

If $y=0$ this simplifies to $-\ln(1-\hat y)$, which tends to zero as $\hat y$ tends to zero, and tends to infinity as $\hat y$ tends to one.

If $y=1$ this simplifies to $-\ln(\hat y)$, which tends to zero as $\hat y$ tends to one, and tends to infinity as $\hat y$ tends to zero.

Both of these are as desired.  It's especially special if you're using a sigmoid activation function at the final level; then the derivative of the cost, times the derivative of the activation function, turns out to be exactly $y-\hat y$, which is pretty much perfect (calculus exercise).  The hyperbolic tangent function looks similar, so has similar properties, but doesn't simplify quite as nicely.

The derivative (with respect to $\hat y$) can be easily computed to be the following:

$$\frac{\partial C}{\partial \hat y}=-\frac{y}{\hat y}+\frac{1-y}{1-\hat y}$$

When we code it, we need to make sure that the zeros properly cancel their respective terms.  Also, as before, we take the mean across all the rows to get the aggregate:

In [2]:
def cost_CE(y_hat, y, diff=False, aggregate=False):
    # Assumes y consists entirely of zeros and ones!
    if diff:
        deriv = np.where(y == 0, 1/(1-y_hat), (-1)/y_hat)
        return deriv
    elif aggregate:
        cost = np.where(y == 0, -np.log(1-y_hat), -np.log(y_hat))
        # sum the errors for each row, then take the mean across all rows
        return np.mean(np.sum(cost, axis=1), axis=0)
    else:
        cost = np.where(y == 0, -np.log(1-y_hat), -np.log(y_hat))
        return cost

Interpret the flags as before.  We will experimentally verify the usefulness of this cost function as soon as our learning system is up and running.

**Bonus reading:** see <a href="http://neuralnetworksanddeeplearning.com/chap3.html">this book</a> (chapter 3, close to the beginning) for an excellent demonstration of why this cost function is superior for classification problems, with animations and so on that I can't really match.

## Trouble with Cross-Entropy

Notice any problems with the above code?  Here's a hint.  First off, we need to assume `y` is always 0 or 1; this isn't a problem, but it should be noted (at least in a comment, and ideally in a docstring).

More importantly, what if `y_hat` breaks out of its range?  We'd like to say `y_hat` should always be strictly between 0 and 1, and if it were a real-valued function, it would be.  However, due to floating point issues, the output of a sigmoid function can easily be exactly one: for example `act_sigmoid(37)==1`.  Zero is more difficult, but `act_sigmoid(-800)==0`.

The problem at one, if you're interested, is that if `np.exp(-z)` is close enough to zero, then when you add one, you get exactly one, because the decimal representation only keeps track of so many digits.  So the sigmoid is just 1/1=1.  The problem with zero only occurs when `np.exp` overflows, but this is a fairly plausible occurrence, especially with rectifer units (which are unbounded).

The situation where the sigmoid comes out to 0 or 1 is not actually a problem -- it's accurate to within any reasonable tolerance.  But it can cause division by zero problems with the fractions above.  The solution is just to clamp the input to a reasonable range; this was chosen to be as close to the bound as is reasonable, without rounding to one:

In [3]:
def cost_CE(y_hat, y, diff=False, aggregate=False):
    # Prevents divide by zero problems
    y_hat = np.clip(y_hat, np.exp(-36), 1-np.exp(-36))
    
    # Assumes y consists entirely of zeros and ones!
    if diff:
        deriv = np.where(y == 0, 1/(1-y_hat), (-1)/y_hat)
        return deriv
    elif aggregate:
        cost = np.where(y == 0, -np.log(1-y_hat), -np.log(y_hat))
        # sum the errors for each row, then take the mean across all rows
        return np.mean(np.sum(cost, axis=1), axis=0)
    else:
        cost = np.where(y == 0, -np.log(1-y_hat), -np.log(y_hat))
        return cost